In [36]:
from keras.preprocessing import image
from keras.models import Model
from keras.applications.vgg16 import VGG16 
from keras.applications.vgg16 import preprocess_input

from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
import numpy as np

import os
import re
print("Loading VGG16 pre-trained model...")
base_model = VGG16(weights='imagenet',input_shape=(150, 150, 3),include_top=False)
model = Model(input=base_model.input,output=base_model.get_layer('block4_pool').output)

Loading VGG16 pre-trained model...


C:\Users\Java\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  


In [37]:


print("[INFO] describing images...")

imagePaths = [os.path.join("C:\\Users\\Java\\Desktop\\knntest",x) for x in os.listdir("C:\\Users\\Java\\Desktop\\knntest")]

labels = []
vggs = []


for (i, imagePath) in enumerate(imagePaths):
    label = re.findall('\D+',imagePath.split(os.path.sep)[-1].split(".")[0])
    img = image.load_img(imagePath, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x).flatten()
    vggs.append(features)
    labels.append(label)
    if i > 0 and i % 1000 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))


labels = np.array(labels)
vggs = np.array(vggs)
print("[INFO] VGG16 matrix: {:.2f}MB".format(vggs.nbytes / (1024 * 1000.0)))

(trainvgg, testvgg, trainvggLabels, testvggLabels) = train_test_split(vggs, labels, test_size=0.25, random_state=42)


print("[INFO] evaluating vgg accuracy...")
model = KNeighborsClassifier(n_neighbors=7,n_jobs=-1)
model.fit(trainvgg, trainvggLabels)
acc = model.score(testvgg, testvggLabels)
print("[INFO] vgg accuracy: {:.2f}%".format(acc * 100))

[INFO] describing images...
[INFO] processed 1000/5167
[INFO] processed 2000/5167
[INFO] processed 3000/5167
[INFO] processed 4000/5167
[INFO] processed 5000/5167
[INFO] VGG16 matrix: 837.05MB
[INFO] evaluating vgg accuracy...


C:\Users\Java\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[INFO] vgg accuracy: 20.43%
